In [ ]:
import Neural_Network_Class as nn
import numpy as np
from tqdm import tqdm
from venn_data import create_venn_data
import matplotlib.pyplot as plt

plt.style.use('dark_background')
plt.rcParams.update({
    "figure.facecolor":  (0.12 , 0.12, 0.12, 1),
    "axes.facecolor": (0.12 , 0.12, 0.12, 1),
})


In [ ]:
# Type 
dense1: nn.Layer_Dense 
activation1: nn.Activation_ReLU
dense2: nn.Layer_Dense
output_activation: nn.Activation_Sigmoid
loss_function: nn.Loss_BinaryCrossentropy
optimizer: nn.Optimizer_Adam

x1_span: np.ndarray
x2_span: np.ndarray
xx1: np.ndarray
xx2: np.ndarray
grid: np.ndarray

In [ ]:
def predict(new_X) -> np.ndarray:
    dense1.forward(new_X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    output_activation.forward(dense2.output)
    predictions = (output_activation.output > 0.5) * 1  # Boolean to int
    return predictions

def distribution(new_X) -> np.ndarray:
    dense1.forward(new_X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    output_activation.forward(dense2.output)
    return output_activation.output

In [ ]:
def calc_z_map() -> np.ndarray:
    Z = predict(grid)
    Z[:, 0] = Z[:, 0] * 2
    Z[:, 1] = Z[:, 1] * 4
    Z[:, 2] = Z[:, 2] * 8
    Z = np.sum(Z, axis=1)
    return Z.reshape(xx1.shape)

def calc_distribs_z_map() -> tuple[np.ndarray]:
    Z = distribution(grid)
    Z1 = Z[:, 0].reshape(xx1.shape)
    Z2 = Z[:, 1].reshape(xx1.shape)
    Z3 = Z[:, 2].reshape(xx1.shape)
    return Z1, Z2, Z3

In [ ]:
def train(X, y, epochs=10001, Z_map_frames=False):
    training_history = {'acc': [], 'loss': [], 'data_loss': [], "regularization_loss": [], "LR": [],
                        "Z_map": [], "distrib_output": [], "Z_map_distrib": []}

    for epoch in tqdm(range(epochs)):
        # Forward pass
        predictions = predict(X)
        accuracy = np.mean(predictions == y)
        
        # Loss calculation
        data_loss = loss_function.calculate(output_activation.output, y)
        regularization_loss = loss_function.regularization_loss(dense1) + \
                              loss_function.regularization_loss(dense2)
        loss = data_loss + regularization_loss

        # Backward pass
        loss_function.backward(output_activation.output, y)
        output_activation.backward(loss_function.dinputs)
        dense2.backward(output_activation.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)

        # Update weights and biases
        optimizer.pre_update_params()
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
        optimizer.post_update_params()

        # Update training history
        training_history['acc'].append(accuracy)
        training_history['loss'].append(loss)
        training_history['data_loss'].append(data_loss)
        training_history['regularization_loss'].append(regularization_loss)
        training_history['distrib_output'].append(output_activation.output)
        training_history['LR'].append(optimizer.current_learning_rate)

    training_history['Z_map'].append(calc_z_map())
    
    return training_history, predictions

# Data

In [ ]:
# Init Layer
dense1 = nn.Layer_Dense(2, 64, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = nn.Activation_ReLU()
dense2 = nn.Layer_Dense(64, 3)
output_activation = nn.Activation_Sigmoid()
loss_function = nn.Loss_BinaryCrossentropy()
optimizer = nn.Optimizer_Adam(learning_rate=0.02, decay=5e-7)

# Data
X, y = create_venn_data(samples=1000, classes=3)

x1_span = np.linspace(min(X[:, 0]) - 0.1, max(X[:, 0]) + 0.1, 500) # 300 is nice
x2_span = np.linspace(min(X[:, 1]) - 0.1, max(X[:, 1]) + 0.1, 500) # 300 is nice
xx1, xx2 = np.meshgrid(x1_span, x2_span)
grid = np.vstack((xx1.ravel(), xx2.ravel())).T # Concatenation des deux matrices ligne par ligne

# Training

In [ ]:
training_history, predictions = train(X, y, epochs=5000)

In [ ]:
print("Training Loss = ", training_history['loss'][-1])
print("Training Accuracy = ", training_history['acc'][-1])

# 2D Graph

In [ ]:
def plot_countour_graph(X, y, predictions, Z):
    # Fix Array
    fix_predictions = predictions.copy()
    fix_predictions[:,0] *= 2
    fix_predictions[:,1] *= 4
    fix_predictions[:,2] *= 8
    fix_y = y.copy()
    fix_y[:,0] *= 2
    fix_y[:,1] *= 4
    fix_y[:,2] *= 8
    # Plot frontiere de décision
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.contourf(xx1, xx2, Z, cmap='coolwarm', alpha=0.5)
    plt.scatter(X[:, 0], X[:, 1], c=np.sum(fix_y, axis=1, keepdims=True), cmap='coolwarm')
    plt.title('Decision boundary of y_true')
    plt.subplot(1, 2, 2)
    plt.contourf(xx1, xx2, Z, cmap='coolwarm', alpha=0.5)
    plt.scatter(X[:, 0], X[:, 1], c=np.sum(fix_predictions, axis=1, keepdims=True), cmap='coolwarm')
    plt.title('Decision boundary of y_pred')

def plot_training_graph(training_history):
    # Plot courbe d'apprentissage
    plt.figure(figsize=(12, 4))
    plt.subplot(2, 3, 1)
    plt.plot(training_history["loss"], label='train loss')
    plt.legend()
    plt.subplot(2, 3, 2)
    plt.plot(training_history["data_loss"], label='train data loss')
    plt.legend()
    plt.subplot(2, 3, 3)
    plt.plot(training_history["regularization_loss"], label='train regularization loss')
    plt.legend()
    plt.subplot(2, 3, 4)
    plt.plot(training_history["acc"], label='train acc')
    plt.legend()
    plt.subplot(2, 3, 5)
    plt.plot(training_history["LR"], label='Learning rate')
    plt.legend()
    plt.show()

def animate_coutour(Z):
    ax.contourf(xx1, xx2, Z, cmap='coolwarm', alpha=0.5, zorder=-1)
    ax.scatter(X[:, 0], X[:, 1], c=y.flatten(), cmap='coolwarm', zorder=1)

In [ ]:
plot_training_graph(training_history)
plot_countour_graph(X, y, predictions, training_history['Z_map'][-1])

# 3D Graph

In [ ]:
import plotly.graph_objects as go

In [ ]:
fix_predictions = predictions.copy()
fix_predictions[:,0] *= 2
fix_predictions[:,1] *= 4
fix_predictions[:,2] *= 8
fix_y = y.copy()
fix_y[:,0] *= 2
fix_y[:,1] *= 4
fix_y[:,2] *= 8

fig = go.Figure(data=[
    go.Scatter3d( 
        x=xx1.flatten(),
        y=xx2.flatten(),
        z=training_history['Z_map'][-1].flatten(),
        mode='markers',
        marker=dict(
            size=1,
            color=training_history['Z_map'][-1].flatten(),                
            colorscale=['blue', 'white', 'red'],  
            opacity=0.2,
            reversescale=False
        )
    ),
    go.Scatter3d( 
        x=X[:, 0].flatten(),
        y=X[:, 1].flatten(),
        z=np.sum(fix_predictions, axis=1),
        mode='markers',
        marker=dict(
            size=1,
            color=np.sum(fix_y, axis=1),                
            colorscale=['blue', 'white', 'red'],  
            opacity=0.8,
            reversescale=False
        )
    )
])

fig.update_layout(template= "plotly_dark", margin=dict(l=0, r=0, b=0, t=0))
fig.layout.scene.camera.projection.type = "orthographic"
fig.show()

In [ ]:
distrib_output_map = calc_distribs_z_map()
distrib_output_X = training_history['distrib_output'][-1].T
distrib_class_1 = distrib_output_map[0]
distrib_class_2 = distrib_output_map[1]
distrib_class_3 = distrib_output_map[2]

calc_distribs_z_map()

fig = go.Figure(data=[
    go.Surface(
        x=xx1,
        y=xx2,
        z=distrib_class_1,
        colorscale='Blues',
        opacity=1,
    ),
    go.Surface(
        x=xx1,
        y=xx2,
        z=distrib_class_2,
        colorscale='Greys',
        opacity=1,
    ),
    go.Surface(
        x=xx1,
        y=xx2,
        z=distrib_class_3,
        colorscale='Reds',
        opacity=1,
    )
])

fig.update_layout(template= "plotly_dark", margin=dict(l=0, r=0, b=0, t=0))
fig.layout.scene.camera.projection.type = "orthographic"
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

fig = make_subplots(rows=1, cols=3, subplot_titles=['Surface 1', 'Surface 2', 'Surface 3'], specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]])

distrib_output_map = calc_distribs_z_map()
distrib_output_X = training_history['distrib_output'][-1].T
distrib_class_1 = distrib_output_map[0]
distrib_class_2 = distrib_output_map[1]
distrib_class_3 = distrib_output_map[2]

fig.add_trace(go.Surface(x=xx1, y=xx2, z=distrib_class_1, colorscale='Blues', opacity=1), row=1, col=1)
fig.add_trace(go.Surface(x=xx1, y=xx2, z=distrib_class_2, colorscale='Greys', opacity=1), row=1, col=2)
fig.add_trace(go.Surface(x=xx1, y=xx2, z=distrib_class_3, colorscale='Reds', opacity=1), row=1, col=3)

fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0))
fig.layout.scene.camera.projection.type = "orthographic"

fig.show()


# Testing with new samples

In [ ]:
def test(X, y):
    test_history = {'acc': 0.0, 'loss':[]}
    
    predictions = predict(X)

    accuracy = np.mean(predictions == y)
    
    loss = loss_function.calculate(output_activation.output, y)

    print(f'accuracy: {accuracy:.3f}, ' +
            f'loss: {loss:.3f}, ')
    
    test_history['acc'] = accuracy
    test_history['loss'] = loss
    
    return test_history, predictions

In [ ]:
# Create test dataset
X_test, y_test = create_venn_data(samples=100, classes=3)

In [ ]:
# Validate the model
test_history, predictions_test = test(X_test, y_test)

If training loss differ from test performance by over ~10%, its a common sign of serious overfitting.

In [ ]:
print("Training Loss = ", training_history['loss'][-1])
print("Test Loss = ", test_history['loss'])

print("Training Accuracy = ", training_history['acc'][-1])
print("Test Accuracy = ", test_history['acc'])

# Compare the training loss and test loss
print(f"{abs(training_history['loss'][-1] - test_history['loss']) / training_history['loss'][-1] * 100:.2f} %")

In [ ]:
plot_countour_graph(X_test, y_test, predictions_test, calc_z_map())
plot_countour_graph(X, y, predictions, calc_z_map())